In [37]:
# La libreria keras es la que se usa por defecto para redes neuronales
# Nota: Es necesario tambien instalar la libreria tensorflow para no tener problemas con el codigo
# Seguramente para una version de python por encima de la 3.6 existan problemas...
# .... para ello usar este comando: pip3 install --upgrade tensorflow-gpu
from keras.layers import Input, Dense
from keras import regularizers, models, optimizers

# Comando adicional, es necesario instalar esta libreria para que trabaje bien
# Dado que optimizers nos ofrece es la segunda version de Adam
#from tensorflow.python.keras.optimizer_v2.adam import Adam
from tensorflow.python.keras import optimizers

from sklearn.decomposition import PCA

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime

# Funciones

# Funcion que sera llamada para realizar los autoencoders lineales

In [38]:
def LinearAE(y, dimension, learning_rate = 1e-4, regularization = 1e-5, epochs=100):
    
    input_layer = Input(shape=(y.shape[1],))
    
    # the encoder layer transforms the input into a representation of size `dimension`
    encoder = Dense(dimension, activation='linear',
                    kernel_regularizer=regularizers.l2(regularization))(input_layer)
    
    # the decoder layer transforms the encoded representation back to the original size
    decoder = Dense(y.shape[1], activation='linear',
                    kernel_regularizer=regularizers.l2(regularization))(encoder)
    
    adam=Adam(lr=learning_rate)
    autoencoder = models.Model(input_layer, decoder)
    autoencoder.compile(optimizer=optimizers.adam_v2.Adam(lr=learning_rate), loss='mean_squared_error')
    
    # the autoencoder aims to map the input back onto 
    # itself: the input data are also the output targets
    autoencoder.fit(y, y, epochs=epochs, batch_size=4, shuffle=True)
    
    return autoencoder

# Funcion para autoencoders no lineales

In [39]:
def DeepAE(y, dimension, learning_rate = 1e-4, regularization = 5e-4, epochs=100):
    input = Input(shape=(y.shape[1],))
    encoded1 = Dense(np.int(2*dimension), activation='tanh',
                    kernel_regularizer=regularizers.l2(regularization))(input)
    encoded2 = Dense(dimension, activation='tanh',
                    kernel_regularizer=regularizers.l2(regularization))(encoded1)
    decoded1 = Dense(np.int(y.shape[1]/2), activation='tanh',
                    kernel_regularizer=regularizers.l2(regularization))(encoded2)
    decoded2 = Dense(y.shape[1], activation='tanh',
                    kernel_regularizer=regularizers.l2(regularization))(decoded1)
    
    autoencoder = models.Model(input, decoded2)
    autoencoder.compile(optimizer=optimizers.adam_v2.Adam(lr=learning_rate), loss='mean_squared_error')
    
    autoencoder.fit(y, y, epochs=epochs, batch_size=4, shuffle=True)
    
    return autoencoder

# Funcion principal

In [40]:
# df: es un dataframe con las columnas que tienen los bonos. La variable de los dias debe ser llamada..
#.. como "Date" en el csv.
def algoritmos_prediccion(df):
    print("organizando el dataset")
    # Transformar el dia a un mejor formato
    df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)
    # Colocar los dias como indice de la base de datos
    df.set_index('Date', drop=True, inplace=True)
    df.index.names = [None]
    print("##########################################")
    print("Dataset organizado, iniciando analisis....")
    #---------------------------------------
    #-------- PCA ------------
    #-----------------------------------
    
    # objeto pca que se crea con tres componentes
    pca_model = PCA(n_components=3, copy=True, whiten=False)
    # Se corre el PCA para el data set
    pca_model.fit(df)
    # Proyectar el dataset en cada uno de los componentes principales
    df_pca = pd.DataFrame(pca_model.transform(df), index=df.index)
    
    # La variable "df_pca_recon" guardara las predicciones arrojadas por pca
    df_pca_recon = pd.DataFrame(pca_model.inverse_transform(df_pca), index=df.index, columns=df.columns)
    
    print("##########################################")
    print("PCA TERMINADO....")
    print("INICIANDO AUTOENCODERS LINEALES")
    #--------------------------------------------
    #-----------AUTOENCODERS LINEALES----------
    #--------------------------------------------
    
    # Llamamos la funcion que crea el autoencoder del dataset
    # podemos variar los parametros: El "3" son los nodos.
    ae_model = LinearAE(df.values, 3, regularization=0, epochs=1000)
    # Los pesos resultantes (nota: si son de interes podriamos retornarlos)
    (w1, b1, w2, b2) = ae_model.get_weights()
    
    # La variable "df_ae_recon" guarda en data frame las predicciones por autoencoder lineal
    df_ae_recon = ae_model.predict(df.values)
    print("##########################################")
    print("AUTOENCODERS LINEALES TERMINADOS ....")
    print("INICIANDO AUTOENCODERS PROFUNDOS")
    #--------------------------------------------
    #-----------AUTOENCODERS PROFUNDOS----------
    #--------------------------------------------
    
    # Llamando a la funcion de autoencoders profundos
    deepAE_model = DeepAE(df, 3, regularization=0, epochs=1000)
    
    # La variable "df_deep_recon" guarda en data frame las predicciones por autoencoder profundo
    df_deep_recon = deepAE_model.predict(df)
    
    print("##########################################")
    print("TERMINADO ANALISIS....")
    
    return df_pca_recon,df_ae_recon,df_deep_recon

In [41]:
df = pd.read_csv('base_yields_definitiva_1.csv', sep=',')

In [42]:
pca_resultado,auto_lineal,auto_profundo=algoritmos_prediccion(df)

organizando el dataset
##########################################
Dataset organizado, iniciando analisis....
##########################################
PCA TERMINADO....
INICIANDO AUTOENCODERS LINEALES
Epoch 1/1000
75/75 [==============================] - 0s 1000us/step - loss: 46.5447
Epoch 2/1000
75/75 [==============================] - 0s 1ms/step - loss: 42.9008
Epoch 3/1000
75/75 [==============================] - 0s 1ms/step - loss: 36.5614
Epoch 4/1000
75/75 [==============================] - 0s 1ms/step - loss: 38.9372
Epoch 5/1000
75/75 [==============================] - 0s 1ms/step - loss: 34.1977
Epoch 6/1000
75/75 [==============================] - 0s 1ms/step - loss: 31.1059
Epoch 7/1000
75/75 [==============================] - 0s 2ms/step - loss: 29.3355
Epoch 8/1000
75/75 [==============================] - 0s 1ms/step - loss: 25.9006
Epoch 9/1000
75/75 [==============================] - 0s 1ms/step - loss: 23.2365
Epoch 10/1000
75/75 [==============================] - 0s

75/75 [==============================] - 0s 2ms/step - loss: 0.2069
Epoch 97/1000
75/75 [==============================] - 0s 973us/step - loss: 0.1907
Epoch 98/1000
75/75 [==============================] - 0s 1ms/step - loss: 0.1729
Epoch 99/1000
75/75 [==============================] - 0s 1ms/step - loss: 0.1680
Epoch 100/1000
75/75 [==============================] - 0s 851us/step - loss: 0.1733
Epoch 101/1000
75/75 [==============================] - 0s 865us/step - loss: 0.1622
Epoch 102/1000
75/75 [==============================] - 0s 919us/step - loss: 0.1641
Epoch 103/1000
75/75 [==============================] - 0s 905us/step - loss: 0.1649
Epoch 104/1000
75/75 [==============================] - 0s 865us/step - loss: 0.1524
Epoch 105/1000
75/75 [==============================] - 0s 838us/step - loss: 0.1557
Epoch 106/1000
75/75 [==============================] - 0s 1ms/step - loss: 0.1426
Epoch 107/1000
75/75 [==============================] - 0s 1ms/step - loss: 0.1410
Epoch 10

75/75 [==============================] - 0s 892us/step - loss: 0.0386
Epoch 194/1000
75/75 [==============================] - 0s 946us/step - loss: 0.0403
Epoch 195/1000
75/75 [==============================] - 0s 865us/step - loss: 0.0353
Epoch 196/1000
75/75 [==============================] - 0s 892us/step - loss: 0.0392
Epoch 197/1000
75/75 [==============================] - 0s 851us/step - loss: 0.0340
Epoch 198/1000
75/75 [==============================] - 0s 878us/step - loss: 0.0383
Epoch 199/1000
75/75 [==============================] - 0s 892us/step - loss: 0.0330
Epoch 200/1000
75/75 [==============================] - 0s 865us/step - loss: 0.0335
Epoch 201/1000
75/75 [==============================] - 0s 946us/step - loss: 0.0353
Epoch 202/1000
75/75 [==============================] - 0s 865us/step - loss: 0.0315
Epoch 203/1000
75/75 [==============================] - 0s 865us/step - loss: 0.0303
Epoch 204/1000
75/75 [==============================] - 0s 865us/step - loss: 0.

75/75 [==============================] - 0s 865us/step - loss: 0.0276
Epoch 290/1000
75/75 [==============================] - 0s 851us/step - loss: 0.0293
Epoch 291/1000
75/75 [==============================] - 0s 851us/step - loss: 0.0262
Epoch 292/1000
75/75 [==============================] - 0s 1ms/step - loss: 0.0293
Epoch 293/1000
75/75 [==============================] - 0s 878us/step - loss: 0.0273 0s - loss: 0.027
Epoch 294/1000
75/75 [==============================] - 0s 1ms/step - loss: 0.0266
Epoch 295/1000
75/75 [==============================] - 0s 851us/step - loss: 0.0368
Epoch 296/1000
75/75 [==============================] - 0s 932us/step - loss: 0.0333
Epoch 297/1000
75/75 [==============================] - 0s 811us/step - loss: 0.0248
Epoch 298/1000
75/75 [==============================] - 0s 878us/step - loss: 0.0252
Epoch 299/1000
75/75 [==============================] - 0s 838us/step - loss: 0.0303
Epoch 300/1000
75/75 [==============================] - 0s 824us/st

75/75 [==============================] - 0s 946us/step - loss: 0.0243
Epoch 387/1000
75/75 [==============================] - 0s 824us/step - loss: 0.0264
Epoch 388/1000
75/75 [==============================] - 0s 878us/step - loss: 0.0266
Epoch 389/1000
75/75 [==============================] - 0s 932us/step - loss: 0.0259
Epoch 390/1000
75/75 [==============================] - 0s 946us/step - loss: 0.0229
Epoch 391/1000
75/75 [==============================] - 0s 851us/step - loss: 0.0234
Epoch 392/1000
75/75 [==============================] - 0s 1ms/step - loss: 0.0260
Epoch 393/1000
75/75 [==============================] - 0s 1ms/step - loss: 0.0263
Epoch 394/1000
75/75 [==============================] - 0s 838us/step - loss: 0.0286
Epoch 395/1000
75/75 [==============================] - 0s 797us/step - loss: 0.0268
Epoch 396/1000
75/75 [==============================] - 0s 824us/step - loss: 0.0282
Epoch 397/1000
75/75 [==============================] - 0s 838us/step - loss: 0.0211

...

75/75 [==============================] - 0s 1ms/step - loss: 15.0040
Epoch 549/1000
75/75 [==============================] - 0s 2ms/step - loss: 13.7041
Epoch 550/1000
75/75 [==============================] - 0s 1ms/step - loss: 15.5370
Epoch 551/1000
75/75 [==============================] - 0s 1ms/step - loss: 14.9133
Epoch 552/1000
75/75 [==============================] - 0s 1ms/step - loss: 16.5302
Epoch 553/1000
75/75 [==============================] - 0s 2ms/step - loss: 14.5559
Epoch 554/1000
75/75 [==============================] - 0s 1ms/step - loss: 15.8525
Epoch 555/1000
75/75 [==============================] - 0s 1ms/step - loss: 14.2711
Epoch 556/1000
75/75 [==============================] - 0s 1ms/step - loss: 15.4062
Epoch 557/1000
75/75 [==============================] - 0s 1ms/step - loss: 16.2872
Epoch 558/1000
75/75 [==============================] - 0s 1ms/step - loss: 14.8348
Epoch 559/1000
75/75 [==============================] - 0s 1ms/step - loss: 14.3696
Epoch 5

75/75 [==============================] - 0s 2ms/step - loss: 14.4944
Epoch 646/1000
75/75 [==============================] - 0s 2ms/step - loss: 15.3949
Epoch 647/1000
75/75 [==============================] - 0s 2ms/step - loss: 14.8865
Epoch 648/1000
75/75 [==============================] - 0s 1ms/step - loss: 14.4345
Epoch 649/1000
75/75 [==============================] - 0s 2ms/step - loss: 14.0863
Epoch 650/1000
75/75 [==============================] - 0s 1ms/step - loss: 15.2208
Epoch 651/1000
75/75 [==============================] - 0s 1ms/step - loss: 14.9062
Epoch 652/1000
75/75 [==============================] - 0s 2ms/step - loss: 15.1611
Epoch 653/1000
75/75 [==============================] - 0s 3ms/step - loss: 14.4925
Epoch 654/1000
75/75 [==============================] - 0s 2ms/step - loss: 13.6423
Epoch 655/1000
75/75 [==============================] - 0s 2ms/step - loss: 14.6724
Epoch 656/1000
75/75 [==============================] - 0s 2ms/step - loss: 16.0519
Epoch 6

75/75 [==============================] - 0s 1ms/step - loss: 14.2099
Epoch 743/1000
75/75 [==============================] - 0s 1ms/step - loss: 15.2271
Epoch 744/1000
75/75 [==============================] - 0s 2ms/step - loss: 15.1838
Epoch 745/1000
75/75 [==============================] - 0s 2ms/step - loss: 15.2655
Epoch 746/1000
75/75 [==============================] - 0s 1ms/step - loss: 14.7705
Epoch 747/1000
75/75 [==============================] - 0s 2ms/step - loss: 14.7363
Epoch 748/1000
75/75 [==============================] - 0s 1ms/step - loss: 15.1619
Epoch 749/1000
75/75 [==============================] - 0s 1ms/step - loss: 15.5764
Epoch 750/1000
75/75 [==============================] - 0s 1ms/step - loss: 15.0134
Epoch 751/1000
75/75 [==============================] - 0s 1ms/step - loss: 14.9744
Epoch 752/1000
75/75 [==============================] - 0s 1ms/step - loss: 14.8926
Epoch 753/1000
75/75 [==============================] - 0s 1ms/step - loss: 15.2851
Epoch 7

75/75 [==============================] - 0s 2ms/step - loss: 14.2757
Epoch 840/1000
75/75 [==============================] - 0s 2ms/step - loss: 14.7950
Epoch 841/1000
75/75 [==============================] - 0s 2ms/step - loss: 15.3003
Epoch 842/1000
75/75 [==============================] - 0s 2ms/step - loss: 15.1125
Epoch 843/1000
75/75 [==============================] - 0s 2ms/step - loss: 14.7621
Epoch 844/1000
75/75 [==============================] - 0s 2ms/step - loss: 15.3823
Epoch 845/1000
75/75 [==============================] - 0s 2ms/step - loss: 15.7736
Epoch 846/1000
75/75 [==============================] - 0s 2ms/step - loss: 15.3974
Epoch 847/1000
75/75 [==============================] - 0s 2ms/step - loss: 16.4873
Epoch 848/1000
75/75 [==============================] - 0s 2ms/step - loss: 15.1607
Epoch 849/1000
75/75 [==============================] - ETA: 0s - loss: 15.74 - 0s 2ms/step - loss: 15.6380
Epoch 850/1000
75/75 [==============================] - 0s 4ms/step

75/75 [==============================] - 0s 3ms/step - loss: 15.9522
Epoch 937/1000
75/75 [==============================] - 0s 3ms/step - loss: 15.6641
Epoch 938/1000
75/75 [==============================] - 0s 2ms/step - loss: 15.1826
Epoch 939/1000
75/75 [==============================] - 0s 3ms/step - loss: 14.9016: 0s - loss: 14.76
Epoch 940/1000
75/75 [==============================] - 0s 4ms/step - loss: 16.8602
Epoch 941/1000
75/75 [==============================] - 0s 3ms/step - loss: 17.0086
Epoch 942/1000
75/75 [==============================] - 0s 3ms/step - loss: 14.9785
Epoch 943/1000
75/75 [==============================] - 0s 3ms/step - loss: 16.3561
Epoch 944/1000
75/75 [==============================] - 0s 3ms/step - loss: 15.1704
Epoch 945/1000
75/75 [==============================] - 0s 3ms/step - loss: 15.2773
Epoch 946/1000
75/75 [==============================] - 0s 5ms/step - loss: 14.7090
Epoch 947/1000
75/75 [==============================] - 0s 3ms/step - los

In [43]:
pca_resultado

,0.1,0.2,0.3,0.4,0.5,1,2,3,4,5,7,10,15,20,25,30
1985-12-31,5.452082,5.822976,5.888554,5.947717,5.965826,7.475727,7.857619,8.180132,8.440929,8.600375,9.025262,9.097984,9.529430,9.390186,9.548497,9.329720
1986-01-31,5.517390,5.879335,5.939202,5.994211,6.009260,7.514417,7.884493,8.206220,8.469540,8.632561,9.067289,9.148659,9.591790,9.452585,9.612154,9.394117
1986-02-28,5.508764,5.846796,5.901342,5.953129,5.978587,7.403909,7.596313,7.757886,7.882127,7.936801,8.211592,8.193464,8.489766,8.385020,8.492368,8.290075
1986-03-31,5.039357,5.352957,5.409584,5.462986,5.494974,6.807443,6.963819,7.088478,7.182374,7.216492,7.445823,7.418360,7.677205,7.599383,7.694980,7.513859
1986-04-30,4.624495,4.999304,5.091966,5.171297,5.217204,6.584022,6.884759,7.081249,7.218825,7.277739,7.515738,7.480236,7.731782,7.640682,7.750820,7.560055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-31,0.093253,0.035794,0.053516,0.078444,0.127578,0.149063,0.092447,0.102208,0.160623,0.227053,0.377202,0.570104,0.845032,1.026387,1.108388,1.165747
2020-08-31,0.133368,0.056376,0.063077,0.079837,0.120719,0.126683,0.083298,0.126336,0.221326,0.320630,0.526108,0.763622,1.096367,1.274735,1.371092,1.430811
2020-09-30,0.118243,0.045934,0.055261,0.073947,0.116449,0.126596,0.084896,0.124657,0.215124,0.309974,0.507161,0.737519,1.061154,1.239564,1.334079,1.393114
2020-11-30,0.101296,0.033487,0.044535,0.064381,0.105969,0.126605,0.112002,0.174538,0.284297,0.393971,0.612060,0.855329,1.197800,1.371643,1.473347,1.530327
